In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
for X,y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

#Define neural network model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)    
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)
        

Using cpu
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the model parameters

In [10]:
loss_fn = nn.CrossEntropyLoss() #Create loss function 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # Create optimizer

In [14]:
# Create function that trains the model (i.e makes predictions on the training dataset and backpropagates)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        
        #Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Loss: 2.310714  [    0/60000]
Loss: 2.295768  [ 6400/60000]
Loss: 2.270382  [12800/60000]
Loss: 2.264269  [19200/60000]
Loss: 2.244676  [25600/60000]
Loss: 2.217985  [32000/60000]
Loss: 2.234011  [38400/60000]
Loss: 2.183135  [44800/60000]
Loss: 2.184276  [51200/60000]
Loss: 2.169603  [57600/60000]
Test Error: 
 Accuracy: 49.8%, Avg loss: 2.147696 

Epoch 2
-------------------------------
Loss: 2.158529  [    0/60000]
Loss: 2.150285  [ 6400/60000]
Loss: 2.084916  [12800/60000]
Loss: 2.103717  [19200/60000]
Loss: 2.055897  [25600/60000]
Loss: 1.990904  [32000/60000]
Loss: 2.036990  [38400/60000]
Loss: 1.933397  [44800/60000]
Loss: 1.942100  [51200/60000]
Loss: 1.896919  [57600/60000]
Test Error: 
 Accuracy: 54.4%, Avg loss: 1.871465 

Epoch 3
-------------------------------
Loss: 1.904410  [    0/60000]
Loss: 1.875768  [ 6400/60000]
Loss: 1.748240  [12800/60000]
Loss: 1.797260  [19200/60000]
Loss: 1.698666  [25600/60000]
Loss: 1.644640  [32000/600

In [16]:
torch.save(model.state_dict(), "model.pth")
print("Saved model to model.pth")

Saved model to model.pth


In [18]:
#model.pth can now be loaded to a fresh model and be reused
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [22]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
